In [ ]:
Splunk Deployment and Configuration SOP
1. Configure Deployment Client (deploymentclient.conf) on Forwarders

Path: /opt/splunkforwarder/etc/system/local/deploymentclient.conf (or in your app folder like /opt/sf/etc/apps/deployment-client/local/deploymentclient.conf)

[deployment-client]
phoneHomeIntervalInSecs = 60

[target-broker:deploymentServer]
targetUri = 10.142.0.3:8089


Create directory if not exists:

mkdir -p /opt/sf/etc/apps/deployment-client/local
vi /opt/sf/etc/apps/deployment-client/local/deploymentclient.conf


Restart forwarder:

cd /opt/sf/bin
./splunk restart

2. Configure Deployment Server serverclass.conf

Path: /opt/splunk/etc/system/local/serverclass.conf

Example configuration:

[global]
repositoryLocation = $SPLUNK_HOME/etc/deployment-apps
targetRepositoryLocation = /opt/splunkforwarder/etc/apps

[serverClass:hf]
whitelist.0 = 10.142.0.4

[serverClass:hf:app:HF-outputs]
restartSplunkWeb = 0
restartSplunkd = 1
stateOnClient = enabled


Reload deployment server after changes:

cd /opt/splunk/bin
./splunk reload deploy-server -class hf

3. Create Index Configuration (indexes.conf)

Example index config for prod_bank_os:

[prod_bank_os]
homePath = $SPLUNK_DB/prod_bank_os/db
coldPath = $SPLUNK_DB/prod_bank_os/colddb
thawedPath = $SPLUNK_DB/prod_bank_os/thaweddb
maxHotBuckets = 10
maxDataSize = auto_high_volume
frozenTimePeriodInSecs = 6048000
repFactor = 0


Location: Create under /opt/splunk/etc/master-apps/prod_all_indexes/local/indexes.conf

After creating or updating indexes, apply cluster bundle:

cd /opt/splunk/bin
./splunk apply cluster-bundle
./splunk show cluster-bundle-status

4. Configure Inputs on Forwarder (inputs.conf)

Example monitor input for logs:

[monitor:///var/log/tuned/tuned.log]
index = prod_bank_os
sourcetype = prod_tuned_log
disabled = 0
whitelist = \.log$
crcSalt = <SOURCE>
ignoreOlderThan = 7d


Location: /opt/splunk/etc/deployment-apps/prod_bank_inputs/local/inputs.conf

5. Configure Outputs on Forwarder (outputs.conf)

Example:

[tcpout]
defaultGroup = indexergroup

[tcpout:indexergroup]
server = 10.142.0.5:9997,10.142.0.6:9997,10.142.0.7:9997
autoLB = true
useACK = true
maxqueuesize = 500


Location: /opt/splunk/etc/deployment-apps/bankserver-output/local/outputs.conf

6. Configure Clustering on Indexers
6.1 Cluster Master (server.conf)
[clustering]
mode = master
replication_factor = 3
search_factor = 2
pass4SymmKey = apple


Location: /opt/splunk/etc/system/local/server.conf

6.2 Cluster Peers (Indexers)
[clustering]
mode = slave
master_uri = https://10.142.0.4:8089

[replication_port://8080]


Location: /opt/splunk/etc/system/local/server.conf

Restart Splunk on cluster master and peers:

cd /opt/splunk/bin
./splunk restart

7. Initialize Search Head Clustering

On each search head, run:

./splunk init shcluster-config \
-mgmt_uri https://<searchhead_ip>:8089 \
-replication_port 8080 \
-replication_factor 2 \
-conf_deploy_fetch_url https://10.142.0.8:8089

./splunk restart


Example for Search Head 1:

./splunk init shcluster-config -mgmt_uri https://10.142.0.4:8089 -replication_port 8080 -replication_factor 2 -conf_deploy_fetch_url https://10.142.0.8:8089
./splunk restart

8. Assign Search Head Captain (only once)
./splunk bootstrap shcluster-captain "https://10.142.0.4:8089,https://10.142.0.7:8089" -servers_list


Check status:

./splunk show shcluster-status

9. Connect Search Heads to Cluster Master

Run on each search head:

./splunk edit cluster-config -mode searchhead -master_uri https://10.142.0.6:8089 -secret apple

10. Props and Transforms Configuration for Parsing
10.1 props.conf
[prod_tuned_log]
TIME_PREFIX =
SHOULD_LINEMERGE = false
LINE_BREAKER = ([\r\n]+)
MAX_TIMESTAMP_LOOKAHEAD = 30
TIME_FORMAT = %b %d %H:%M:%S
TRUNCATE = 0
TRANSFORMS-null = setnull


Location: /opt/splunk/etc/deployment-apps/all_props/local/props.conf

10.2 transforms.conf
[setnull]
REGEX = .*
DEST_KEY = queue
FORMAT = nullQueue


Location: /opt/splunk/etc/deployment-apps/all_props/local/transforms.conf

11. Role Authorization (authorize.conf)

Example role for bank user with restricted permissions:

[role_bank_user]
importRoles = user
schedule_rtsearch = disable
schedule_search = disable
srchIndexesAllowed = prod_bank_os, prod_cart_os
srchMaxTime = 8640000
rtsearch = disable
srchJobsQuota = 500


Location: /opt/splunk/etc/shcluster/apps/all_authorize/local/authorize.conf

12. User Authentication (authentication.conf)

Example user list:

bank_user = murali;guru


Location: /opt/splunk/etc/shcluster/apps/all_authentication/local/authentication.conf

13. Saved Searches (savedsearches.conf)

Example alert saved search:

[Report_count_greater_than_100]
alert.track = 0
dispatch.earliest_time = -24h@h
dispatch.latest_time = now
display.general.timeRangePicker.show = 0
display.general.type = statistics
display.page.search.tab = statistics
display.visualizations.show = 0
request.ui_dispatch_app = search
request.ui_dispatch_view = search
search = index=prod_bank_os sourcetype=audit_log | stats count by addr | dedup addr | search count>100


Location: /opt/splunk/etc/apps/search/local/savedsearches.conf

14. Change Ownership of Alerts or Dashboards
14.1 Change alert ownership

Edit /opt/splunk/etc/apps/search/metadata/local.meta:

[savedsearches/Report_count_greater_than_100]
access = read : [ * ], write : [ admin ]
export = none
owner = vijay
version = 7.1.1
modtime = 1536474312.829819000


Or via curl:

curl -k -u admin:changeme -d 'owner=foo' -d 'sharing=app' https://localhost:8089/servicesNS/admin/search/saved/searches/Report_count_greater_than_100/acl

14.2 Change dashboard ownership

Edit /opt/splunk/etc/apps/search/metadata/local.meta:

[views/test_dashboard1]
access = read : [ * ], write : [ admin ]
export = none
owner = vijay
version = 7.1.1
modtime = 1536474312.829819000


Or via curl:

curl -k -u admin:changeme -d 'owner=foo' -d 'sharing=app' https://localhost:8089/servicesNS/admin/search/data/ui/views/test_dashboard1/acl

15. Splunk Upgrade Steps

Step 1: Download upgrade package

wget -O splunk-7.1.3-51d9cac7b837-Linux-x86_64.tgz 'https://www.splunk.com/bin/splunk/DownloadActivityServlet?architecture=x86_64&platform=linux&version=7.1.3&product=splunk&filename=splunk-7.1.3-51d9cac7b837-Linux-x86_64.tgz&wget=true'


Step 2: Backup existing etc folder

cd /opt/splunk
tar -zcvf etc.tgz ./etc


Step 3: Stop Splunk

cd /opt/splunk/bin
./splunk stop


Step 4: Extract new version

cd /opt
tar -xvf splunk-7.1.3-51d9cac7b837-Linux-x86_64.tgz


Step 5: Start Splunk with license acceptance

cd /opt/splunk/bin
./splunk start --accept-license

16. Splunk Commands for Troubleshooting

Reset fishbucket for a specific file (example: /etc/csh.login):

./splunk cmd btprobe -d SPLUNK_HOME/var/lib/splunk/fishbucket/splunk_private_db --file /etc/csh.login --reset


Check Splunk logs for TCP output errors:

cat splunkd.log | grep "TcpOutputProc"
cat splunkd.log | grep "TcpOutputFd"

17. Sample Input Types
Script input (inputs.conf)
[script://./bin/my_script.sh]
interval = 86400  ; daily (every 24 hours)
index = <index_name>
sourcetype = <sourcetype_name>
queueSize = 500MB

Syslog input
[tcp://514]
connection_host = 10.142.0.11
index = syslog
sourcetype = linux_audit
disabled = 0
startFrom = oldest
ignoreOlderThan = 2d